In [ ]:
import datetime
import os
import shutil
import pandas as pd
import numpy as np
import tensorflow as tf 
from matplotlib import pyplot as plt
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, DenseFeatures
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
# Now read the CSV files
colums = [
    'fare_amount',
    'pickup_datetime',
    'pickup_longitude',
    'pickup_latitude',
    'dropoff_longitude',
    'dropoff_latitude',
    'passenger_count',
    'key'
]
label_colums = 'fare_amount'
unwanted_coums = ['pickup_datetime','key']
defaults = [[0.0],['na'],[0.0],[0.0],[0.0],[0.0],[0.0],['na']]
def feature_and_labels(row_data):
  label = row_data.pop(label_colums)
  features = row_data
  for unwanted_coloms in unwanted_coums:
    features.pop(unwanted_coloms)
  return features, label
def create_dataset(pattern,batch_size=1,mode='eval'):
  dataset = tf.data.experimental.make_csv_dataset(pattern,batch_size,colums,defaults)
  dataset = dataset.map(feature_and_labels)
  if mode == 'train':
    dataset = dataset.shuffle(buffer_size=1000).repeat()
  dataset = dataset.prefetch(1)
  return dataset

In [ ]:
INPUT_COLS = [
    'pickup_longitude',
    'pickup_latitude',
    'dropoff_longitude',
    'dropoff_latitude',
    'passenger_count',
]
features_coloms = {colname: tf.feature_column.numeric_column(colname)
                    for colname in INPUT_COLS
                   }

In [ ]:
#Building a simple sequential model
model = Sequential([
                   DenseFeatures(feature_columns= features_coloms.values()),
                   Dense(units = 32, activation='relu',name='h1'),
                   Dense(units=8, activation='relu', name='h2'),
                   Dense(units=1, activation='linear', name='ouput')
])

In [ ]:
#create a custom evaluation matric
def rmse(y_pred,y_true):
  return tf.sqrt(tf.reduce_mean(tf.square(y_pred-y_true)))
model.compile(optimizer='adam',loss='mse',metrics=[rmse,'mse'])

In [ ]:
train_batch_size = 1000
train_example = 10000 * 5
num_eval = 50
num_eval_example = 10000
train_data = create_dataset('/content/drive/MyDrive/Colab Notebooks/Data sets/05_Tensorflow_Keras_Sequential/taxi-train.csv',
                            batch_size=train_batch_size,
                            mode = 'train')
eval_data = create_dataset('/content/drive/MyDrive/Colab Notebooks/Data sets/05_Tensorflow_Keras_Sequential/taxi-valid.csv',
                           batch_size=1000,
                           mode = 'eval')


In [ ]:
%time
setps_per_epoch = train_example // (train_batch_size * num_eval)
logdir = './taxi_trained'
history = model.fit(x=train_data,
                    steps_per_epoch = setps_per_epoch,
                    epochs = num_eval,
                    validation_data = eval_data,
                    callbacks=[TensorBoard(log_dir=logdir)])

CPU times: user 1 µs, sys: 1e+03 ns, total: 2 µs
Wall time: 4.53 µs
Epoch 1/50
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
1/1 [==============================] - ETA: 0s - loss: 759.1771 - rmse: 27.5532 - mse: 759.1771WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'collections.OrderedDict'> input: OrderedDict([('pickup_longitude', <tf.Tensor 'ExpandDims_4:0' shape=(1000, 1) dtype=float32>), ('pickup_latitude', <tf.Tensor 'ExpandDims_3:0' shape=(1000, 1) dtype=float32>), ('dropoff_longitude', <tf.Tensor 'ExpandDims_1:0' shape=(1000, 1) dtype=float32>), ('dropoff_latitude', <tf.Tensor 'ExpandDims:0' shape=(1000, 1) dtype=float32>), ('passenger_count', <tf.Tensor 'ExpandDims_2:0' shape=(1000, 1) dtype=float32>)])
Consider rewriting this model with the Functional API.
